In [ ]:
# --- COLAB SETUP CELL ---
import os
import sys

USER = "ever-oli"
REPO = "VERITAS"
BRANCH = "main"

if not os.path.exists(REPO):
    !git clone https://github.com/{USER}/{REPO}.git
else:
    %cd {REPO}
    !git pull
    %cd ..

repo_path = os.path.abspath(REPO)
if repo_path not in sys.path:
    sys.path.append(repo_path)

print("Installing dependencies (PyTorch-based, no TensorFlow)...")

# Core audio/ML deps
!pip install -q librosa scipy matplotlib seaborn tqdm soundfile transformers pretty_midi

# PyTorch (already in Colab usually)
!pip install -q torch --index-url https://download.pytorch.org/whl/cu118

# Demucs
!pip install -q demucs

# Basic Pitch PyTorch (no TF!)
!pip install -q git+https://github.com/gudgud96/basic-pitch-torch.git

print(f"✓ Setup complete. Repo path: {repo_path}")

/content/VERITAS
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 6 (delta 2), reused 6 (delta 2), pack-reused 0 (from 0)
Unpacking objects: 100% (6/6), 3.45 KiB | 3.45 MiB/s, done.
From https://github.com/ever-oli/VERITAS
   81345e0..8195f3e  main       -> origin/main
Updating 81345e0..8195f3e
Fast-forward
 models/audio_to_ppr.py     | 101 +++++++++++++++++++++++---
 notebooks/week1_test.ipynb | 172 +++++++++++++++++++++++++++++++++++++++++++++
 2 files changed, 265 insertions(+), 8 deletions(-)
 create mode 100644 notebooks/week1_test.ipynb
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 6 (delta 2), reused 6 (delta 2), pack-reused 0 (from 0)
Unpacking objects: 100% (6/6), 3.45 KiB | 3.45 MiB/s, done.
From https://github.com/ever-oli/VERITAS
   81345e0..8195f3e  main       -> origin/main
Upda

In [ ]:
import os

test_audio = os.path.join("VERITAS", "test_jazz.mp3")
!wget -O {test_audio} https://files.freemusicarchive.org/storage-freemusicarchive-org/music/WFMU/Broke_For_Free/Directionless_EP/Broke_For_Free_-_01_-_Night_Owl.mp3
print(f"Saved to: {os.path.abspath(test_audio)}")

In [1]:
import onnxruntime as ort
import numpy as np
import librosa

class NeuralNoteInference:
    def __init__(self, model_path="path/to/features_model.onnx"):
        self.session = ort.InferenceSession(model_path)
        # TODO: load CNN weights if needed
    
    def extract_ppr(self, audio_path: str) -> np.ndarray:
        """
        Args:
            audio_path: Path to audio file
        Returns:
            (Time, 88) numpy array of note probabilities
        """
        # TODO: load audio, preprocess, run inference, postprocess
        pass

ModuleNotFoundError: No module named 'onnxruntime'

In [ ]:
import os
import sys

# Ensure repo path is on sys.path (in case cell 1 wasn't re-run)
repo_path = os.path.abspath("VERITAS")
if repo_path not in sys.path:
    sys.path.append(repo_path)

from models.audio_to_midi_image import AudioToMidiImage

# Path to the downloaded test audio
test_audio = os.path.join("VERITAS", "test_jazz.mp3")

# Initialize pipeline
pipeline = AudioToMidiImage()

try:
    # 1. Separate ALL stems (vocals, drums, bass, other)
    print("Step 1: Running Demucs stem separation...")
    stems = pipeline.separate_stems(test_audio)
    print(f"Generated stems: {list(stems.keys())}")
    
    # 2. Generate MIDI and images for each stem
    print("\nStep 2: Converting each stem to MIDI + piano roll images...")
    results = pipeline.stems_to_midi(stems)
    
    print("\n✅ Success! Multi-stem MIDI + image generation complete.")
    print(f"Results saved to: {os.path.dirname(list(results.values())[0]['midi'])}")
    
    # Show what was generated
    for stem_name, paths in results.items():
        print(f"- {stem_name}: MIDI + Image generated")
    
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()

AudioToPPR initialized on cuda
Step 1: Running Demucs...
Error: 'AudioToPPR' object has no attribute 'separate_stems'




**Why This Approach:**
- Uses NeuralNote's optimized models directly (no TF hell)
- Gets the visual output you want for VLM
- Simple, plug-and-play with existing pipeline

**If NeuralNote's models are too C++-heavy**, suggest the next best: `piano-transcription-inference` (pure PyTorch, outputs MIDI, easy to render).

Focus on **getting a working pipeline end-to-end**. I can refine once I have basic transcription + visualization.

---

**Why This Prompt is Better:**
1. **Focuses on NeuralNote's models** - directly from `Lib/ModelData/`, as you specified
2. **Simple output** - MIDI → image, no matrices
3. **Practical** - Colab cells that work
4. **Aligns with your goal** - piano roll image for VLM/MusicLM

Send this to Gemini. They should come back with code that loads NeuralNote's models and renders the piano roll image.